<a href="https://colab.research.google.com/github/sooramahesh/NLP_LAB_ASSIGNMENTS/blob/main/2203A51159_SooraMahesh_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Example data with added start and end tokens in French translations
data = [
    ("hello", "<start> bonjour <end>"),
    ("how are you", "<start> comment ça va <end>"),
    ("I am fine", "<start> je vais bien <end>"),
    ("what is your name", "<start> comment tu t'appelles <end>"),
    ("my name is", "<start> je m'appelle <end>"),
    ("thank you", "<start> merci <end>"),
    ("goodbye", "<start> au revoir <end>")
]

# Separate data into English and French sentences
english_texts, french_texts = zip(*data)

# Tokenize and create vocabulary
def tokenize(texts):
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, padding='post')
    return padded_sequences, tokenizer

# Tokenize English and French texts
english_sequences, english_tokenizer = tokenize(english_texts)
french_sequences, french_tokenizer = tokenize(french_texts)

# Define vocabulary sizes
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(english_vocab_size, 64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(64, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(french_vocab_size, 64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(french_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Shift French sequences by one for decoder target data
decoder_input_data = french_sequences[:, :-1]
decoder_target_data = french_sequences[:, 1:]
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# Training the model
model.fit([english_sequences, decoder_input_data], decoder_target_data, batch_size=2, epochs=500)

# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Translation function
def translate_sentence(input_sentence):
    # Preprocess input sentence
    input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
    input_sequence = pad_sequences(input_sequence, maxlen=english_sequences.shape[1], padding='post')

    # Encode the input
    states_value = encoder_model.predict(input_sequence)

    # Generate translation
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = french_tokenizer.word_index['<start>']  # Start token for French
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = french_tokenizer.index_word.get(sampled_token_index, '')

        # Stop if end of sentence is reached
        if sampled_word == '<end>' or len(decoded_sentence) > 20:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        # Update the target sequence and states
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# Test translation on a new sentence
print(translate_sentence("hello"))


Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 2.7683
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.7262 
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.6983  
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.6427 
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.6081 
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.5464 
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.3065  
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.2794 
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.1376 
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.9964 
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.0631  
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.9493 
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.0503  
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.9017 
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.0437 
Epoch 16/500
4/